In [1]:
import cv2

cap = cv2.VideoCapture('./videos/drummer.mp4')

while cap.isOpened():
    
    ret, frame = cap.read()
    
    # if frame is read correctly ret is True
    if not ret:
        print("Stream stopped.")
        break

    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) == ord('q'):
        break
    
        
cap.release()
cv2.destroyAllWindows()

Stream stopped.


: 

# Display video from Notebook

In [2]:
import cv2
from IPython.display import display, Image, clear_output
import numpy as np

# Open the video file
cap = cv2.VideoCapture('./videos/drummer.mp4')

if not cap.isOpened():
    print("Cannot open video file")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("Stream stopped.")
        break

    # Convert the frame to JPEG format for display in Jupyter
    _, buffer = cv2.imencode('.jpg', frame)
    img_bytes = buffer.tobytes()

    # Display the frame in Jupyter Notebook
    display(Image(data=img_bytes))
    clear_output(wait=True)  # Clear previous frame for smoother playback

    # Optional: Use a break condition if needed
    # To stop manually, you can stop the kernel or add conditions here.
    
cap.release()
print("Video stream ended.")

Stream stopped.
Video stream ended.


## ```cap.get(id)``` 
Is a method where id is a number from 0 to 18. Each number denotes a property of the video (if it is applicable to that video)

In [3]:
import cv2

cap = cv2.VideoCapture('./videos/drummer.mp4')

for i in range(0,18):
    print(cap.get(i))

0.0
0.0
3.3333333333333335e-05
640.0
360.0
30.0
828601953.0
466.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


**You can see index 3 and 4 correspond to the video dimensions**

In [ ]:
import cv2
import numpy as np
import torch
from facenet_pytorch import MTCNN

# Initialize the MTCNN model for face detection
mtcnn = MTCNN(keep_all=True, device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))

# Our sketch generating function for the detected face
def sketch(image):
    # Convert image to grayscale
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Clean up image using Gaussian Blur
    img_gray_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    
    # Extract edges
    canny_edges = cv2.Canny(img_gray_blur, 10, 70)
    
    # Invert binarize the image
    _, mask = cv2.threshold(canny_edges, 70, 255, cv2.THRESH_BINARY_INV)
    return mask


# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break
    
    # Convert frame to RGB (MTCNN expects RGB images)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detect faces using MTCNN
    boxes, _ = mtcnn.detect(rgb_frame)
    
    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            # Ensure the bounding box is within the frame
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)
            
            # Extract the face region
            face_region = frame[y1:y2, x1:x2]
            
            # Apply sketch effect to the face region
            face_sketch = sketch(face_region)
            
            # Convert sketch to 3 channels to overlay on the original frame
            face_sketch_color = cv2.cvtColor(face_sketch, cv2.COLOR_GRAY2BGR)
            
            # Replace the face region in the original frame with the sketch
            frame[y1:y2, x1:x2] = cv2.resize(face_sketch_color, (x2 - x1, y2 - y1))
    
    # Display the frame with face sketches
    cv2.imshow('Live Sketch with Face Detection', frame)
    
    # Break loop on pressing 'Enter'
    if cv2.waitKey(1) == 13:  # 13 is the Enter Key
        break

# Release camera and close windows
cap.release()
cv2.destroyAllWindows()

2024-12-01 13:45:58.046 python[38991:4475326] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-12-01 13:46:00.571 python[38991:4475326] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-01 13:46:00.571 python[38991:4475326] +[IMKInputSession subclass]: chose IMKInputSession_Modern
